In [1]:
from mqt.yaqs.core.data_structures.networks import MPS, MPO
from mqt.yaqs import simulator
from mqt.yaqs.core.data_structures.simulation_parameters import AnalogSimParams, Observable
from mqt.yaqs.core.libraries.gate_library import X, Z, RuntimeCost
from mqt.yaqs.core.data_structures.noise_model import NoiseModel

import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
def tdvp_simulator(H_0, noise_model, state=None):
    L = H_0.length

    # Ising
    state = MPS(length=L)

    # Heisenberg
    # elapsed_time = 2
    # state = MPS(length=L, state="Neel")

    measurements = [Observable(Z(), [0])] + [Observable("entropy", [L//2, L//2+1])] + [Observable("max_bond")]
    sim_params = AnalogSimParams(observables=measurements,
                                elapsed_time=5,
                                dt=0.1,
                                num_traj=10,
                                threshold=1e-6,
                                trunc_mode="discarded_weight",
                                order=2,
                                sample_timesteps=False)

    simulator.run(state, H_0, sim_params, noise_model=noise_model)
    print("Obs Exp Val", sim_params.observables[0].results[-1])
    print("Entropy", sim_params.observables[1].results[-1])
    print("Max Bond", sim_params.observables[2].results[-1])
    return sim_params.observables[1::]

In [3]:
L_list = [8, 12, 16, 20, 24, 28, 32, 40, 48, 64]
for L in L_list:
    print(L)
    J = 1
    h = 1
    H_0 = MPO()
    H_0.init_ising(L, J, h)

    dps = np.logspace(-3, 0, 10)
    results = []
    for j, dp in enumerate(dps):
        print(j+1, "of", len(dps))
        gamma = dp/0.1  # assuming dt=0.1
        # Unraveling 1
        # noise_model = NoiseModel([
        #     {"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["pauli_z", "pauli_x", "pauli_y"]
        #     ])
        # Unraveling 2
        noise_model = NoiseModel([
            {"name": name, "sites": [i], "strength": gamma} for i in range(L) for name in ["measure_0", "measure_1", "measure_x_0", "measure_x_1", "measure_y_0", "measure_y_1"]
        ])
        gamma = 2*gamma
        cost = tdvp_simulator(H_0, noise_model)
        results.append(cost)

    filename = f"u2t1_{L}.pickle"
    with open(filename, 'wb') as handle:
        pickle.dump(results, handle)

8
1 of 10


Running trajectories: 100%|█████████████████████| 10/10 [00:01<00:00,  5.61it/s]


Obs Exp Val (-0.10818472472931298+0j)
Entropy (1.555112491534267+0j)
Max Bond (8+0j)
2 of 10


Running trajectories: 100%|█████████████████████| 10/10 [00:01<00:00,  5.73it/s]


Obs Exp Val (-0.045696502517596466+0j)
Entropy (1.4464983439620784+0j)
Max Bond (7.7+0j)
3 of 10


...

Obs Exp Val (-0.06283551643721631+0j)
Entropy (3.0209153906111115+0j)
Max Bond (53.7+0j)
9 of 10


Running trajectories: 100%|█████████████████████| 10/10 [00:48<00:00,  4.83s/it]


Obs Exp Val (0.11601330694852569+0j)
Entropy (3.093336597591949+0j)
Max Bond (55.7+0j)
10 of 10


Running trajectories: 100%|█████████████████████| 10/10 [00:51<00:00,  5.16s/it]

Obs Exp Val (-0.09116906519307838+0j)
Entropy (3.3749699614121837+0j)
Max Bond (57.800000000000004+0j)
